In [28]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import faiss
import os, io, pickle
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    SpacyTextSplitter
)
from langchain.vectorstores import FAISS
import numpy as np
import pandas as pd
import tiktoken
from sentence_transformers import SentenceTransformer
import shutil
import statistics
import os
from dotenv import load_dotenv

In [29]:
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

def authenticate_google():
    creds = None
    if os.path.exists("token"):
        with open("token", "rb") as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials3.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open("token", "wb") as token:
            pickle.dump(creds, token)
    
    return build("drive", "v3", credentials=creds)

def download_pdf(file_id, output_path):
    service = authenticate_google()
    request = service.files().get_media(fileId=file_id)
    fh = io.FileIO(output_path, "wb")
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print(f"Téléchargement : {int(status.progress() * 100)}%")
    print(f"Fichier téléchargé : {output_path}")


file_id = "1AHE1lXi_kyrtRw31qEGJ7OYE9EO8kfGe"
download_pdf(file_id, "Histoire_CM1.pdf")

Téléchargement : 100%
Fichier téléchargé : Histoire_CM1.pdf


In [30]:
# Fonction pour estimer le nombre de tokens d’un texte
def count_tokens(text, model="gpt-3.5-turbo"):
    enc = tiktoken.encoding_for_model(model)
    return len(enc.encode(text))

In [31]:
# Charger ton PDF
loader = PyPDFLoader("Histoire_CM1.pdf",
                     mode = "page", # Extract the PDF by page. Each page is extracted as a langchain Document object
                     # mode = "single" # PyPDFLoader will split the PDF as a single text flow
                     )
docs = loader.load()

In [32]:
full_text = "\n".join([page.page_content for page in docs])
full_text

'LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  \n \n \n1)  Qu’est-ce que l’Histoire ? \n \n*L’Histoire est l’étude de notre passé  pour mieux \ncomprendre notre vie aujourd’hui. \n \n *Pour découvrir notre passé, les historiens font des fouilles \narchéologiques, étudient des objets, des documents, des récits…  \n*Ils représentent le temps par une ligne graduée  : c’est la frise \nchronologique.  \n \n*Avant l’invention de l’écriture, c’est la Préhistoire , ensuite vient \nl’Histoire.  \n*L’Histoire de France est divisée en 5 périodes : \nl’Antiquité – le Moyen Âge – les Temps Modernes – le XIX ème \nsiècle – le XXème siècle. \n \n \n \n \n2) Des traces du passé : les grottes ornées \n \n*En 1940, 4 enfants découvrent une grotte recouverte \nde peintures  : des taureaux, des cerfs, des chevaux…  : \nla grotte de Lascaux.  En datant les objets trouvés dans la grotte on \nsait qu’elle a été peinte il y a environ 17000 ans.  \n \n*En 1994, Jean -Marie Chauvet découvre une autre grotte pei

In [33]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_text(full_text)
token_counts = [count_tokens(chunk) for chunk in chunks]

print(f"Nombre de chunks : {len(chunks)}")
print(f"Tokens/chunk (moy) : {round(statistics.mean(token_counts))}")
print(f"Tokens max : {max(token_counts)}")
print(f"Aperçu du 1er chunk :\n{chunks[0]}")


Nombre de chunks : 27
Tokens/chunk (moy) : 144
Tokens max : 161
Aperçu du 1er chunk :
LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  
 
 
1)  Qu’est-ce que l’Histoire ? 
 
*L’Histoire est l’étude de notre passé  pour mieux 
comprendre notre vie aujourd’hui. 
 
 *Pour découvrir notre passé, les historiens font des fouilles 
archéologiques, étudient des objets, des documents, des récits…  
*Ils représentent le temps par une ligne graduée  : c’est la frise 
chronologique.  
 
*Avant l’invention de l’écriture, c’est la Préhistoire , ensuite vient 
l’Histoire.


In [34]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks)

print("Taille des embeddings : ", embeddings.shape)
for i, (chunk, emb) in enumerate(zip(chunks, embeddings)):
    print("="*60)
    print(f"Chunk {i+1}:")
    print(chunk)
    print(f"\n Embedding (taille {len(emb)}):")
    print(np.round(emb[:10], 3))  # Affiche les 10 premières valeurs, arrondies pour lisibilité
    print("="*60)

/usr/local/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Taille des embeddings :  (27, 384)
Chunk 1:
LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  
 
 
1)  Qu’est-ce que l’Histoire ? 
 
*L’Histoire est l’étude de notre passé  pour mieux 
comprendre notre vie aujourd’hui. 
 
 *Pour découvrir notre passé, les historiens font des fouilles 
archéologiques, étudient des objets, des documents, des récits…  
*Ils représentent le temps par une ligne graduée  : c’est la frise 
chronologique.  
 
*Avant l’invention de l’écriture, c’est la Préhistoire , ensuite vient 
l’Histoire.

 Embedding (taille 384):
[-0.024  0.15   0.059 -0.052 -0.017  0.151 -0.052  0.037 -0.035  0.003]
Chunk 2:
l’Histoire.  
*L’Histoire de France est divisée en 5 périodes : 
l’Antiquité – le Moyen Âge – les Temps Modernes – le XIX ème 
siècle – le XXème siècle. 
 
 
 
 
2) Des traces du passé : les grottes ornées 
 
*En 1940, 4 enfants découvrent une grotte recouverte 
de peintures  : des taureaux, des cerfs, des chevaux…  : 
la grotte de Lascaux.  En datant les objets trouvés d

In [35]:
chroma_db_path = "chroma_db"
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
documents = [Document(page_content=chunk, metadata={"chunk_id": i}) for i, chunk in enumerate(chunks)]


chroma_db = Chroma.from_documents(
    documents, 
    embedding_model, 
    persist_directory = chroma_db_path)

chroma_db

In [36]:
# Test de recherche
query = "Qui est Napoléon ?"

# Récupère les chunks les plus pertinents associés à la requête
# Exemple avec une recherche basée sur un score de similarité
retriever = chroma_db.as_retriever(
    search_type = "similarity_score_threshold",
    search_kwargs = {"k": 3, "score_threshold": 0.2},
)

relevant_docs = retriever.invoke(query)
relevant_docs

/usr/local/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[Document(id='29bbe0f9-b398-47bd-b984-47c56115916a', metadata={'chunk_id': 23}, page_content='autre gouvernement est mis en place.  \n \n*Il sera renversé par Napoléon le 18 brumaire.   \n \n  *Constitution : texte qui précise comment un pays est dirigé.\n21) Le Consulat et l’Empire (1799-1815) \n \n*Le général Bonaparte remporte de nombreuses victoires \nmilitaires, comme Austerlitz.  \n \n*Il  adopte une nouvelle constitution qui lui donne tous les \npouvoirs.  \n \n*Il crée les départements dirigés par un préfet, la banque de France, \nles lycées…'),
 Document(id='4371df77-b70a-42f9-8f34-f72655c85ab3', metadata={'chunk_id': 23}, page_content='autre gouvernement est mis en place.  \n \n*Il sera renversé par Napoléon le 18 brumaire.   \n \n  *Constitution : texte qui précise comment un pays est dirigé.\n21) Le Consulat et l’Empire (1799-1815) \n \n*Le général Bonaparte remporte de nombreuses victoires \nmilitaires, comme Austerlitz.  \n \n*Il  adopte une nouvelle constitution qui lui 

In [37]:
print("Nombre de chunks pertinents :", len(relevant_docs))
    
# Affiche les résultats pertinents avec les métadonnées associées
print("\n--- Documents les plus pertinents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")
    if doc.metadata:
        print(f"chunk_id : {doc.metadata.get('chunk_id')}\n")

Nombre de chunks pertinents : 3

--- Documents les plus pertinents ---
Document 1:
autre gouvernement est mis en place.  
 
*Il sera renversé par Napoléon le 18 brumaire.   
 
  *Constitution : texte qui précise comment un pays est dirigé.
21) Le Consulat et l’Empire (1799-1815) 
 
*Le général Bonaparte remporte de nombreuses victoires 
militaires, comme Austerlitz.  
 
*Il  adopte une nouvelle constitution qui lui donne tous les 
pouvoirs.  
 
*Il crée les départements dirigés par un préfet, la banque de France, 
les lycées…

chunk_id : 23

Document 2:
autre gouvernement est mis en place.  
 
*Il sera renversé par Napoléon le 18 brumaire.   
 
  *Constitution : texte qui précise comment un pays est dirigé.
21) Le Consulat et l’Empire (1799-1815) 
 
*Le général Bonaparte remporte de nombreuses victoires 
militaires, comme Austerlitz.  
 
*Il  adopte une nouvelle constitution qui lui donne tous les 
pouvoirs.  
 
*Il crée les départements dirigés par un préfet, la banque de France, 
les

# MON TEST

In [38]:
context = chroma_db.get()
context

{'ids': ['6c7d9a59-9410-402d-b3bc-cf0ea77426b2',
  '5dacb751-c7bf-44f5-9b33-87bb2430b5c7',
  '98795d97-b941-45bf-8946-bb2fe8a78aec',
  '22953602-b22f-42e6-a235-e4fd82cc6608',
  '722d071e-03ea-4900-bef2-57e844848c64',
  '5c635ca9-0e2a-4f32-831c-300496169891',
  'a8ea1a21-8209-4a89-b618-4dd72d3c74ed',
  'ed7168a6-5a1a-4ed0-b4e7-23b82a4483ac',
  '6c5d2f0b-7e10-447c-8bb4-a8173cbb92cc',
  'bad4196e-6bc6-43af-8b31-0ff97a77c80f',
  'e60b504d-e8e8-4d5e-913e-23504b533ab9',
  '7803982f-1d10-470a-b5f6-fff5b8eb70f7',
  '9b458700-5536-4be9-ac36-8b2eb155e324',
  '7f449713-ffa8-4c32-a868-a1b5a399a268',
  'e7ddaac9-5808-472e-a4d7-0d055a46d302',
  'dde38e43-b583-434a-bb37-dd6eb6a41344',
  'cd8d240c-2ef0-4914-bac3-151697754c65',
  '10910847-614e-43dd-9427-4e5356a624d3',
  '374a8e0b-3d46-4720-abd0-6220b2d55597',
  'f2b11d87-fae8-481e-a140-ae38bebdd356',
  'b62527e9-6c07-458d-bce4-d9b25a94d5a0',
  'd21aabee-468a-4bf6-b58a-ec21abb1eeed',
  'ea64c6e2-a115-42fc-b66d-3ca84c99d4b9',
  '29bbe0f9-b398-47bd-b984-

In [39]:
prompt = f"""
Tu es un expert en pédagogie et tu dois créer une question de quiz à choix multiples (QCM) à partir du texte ci-dessous.

**Important** : tu ne dois utiliser que les informations contenues dans le texte ci-dessous.  
**Tu ne dois en aucun cas utiliser des connaissances extérieures.**

Contrainte :
- La question doit porter sur une information claire et importante du texte.
- Il doit y avoir exactement 4 choix.
- Une seule bonne réponse.
- Il faut que QCM porte uniquement
- Retourne ta réponse **en JSON valide** dans ce format :

{{
  "question": "...",
  "choices": ["...", "...", "...", "..."],
  "correct_answer": "..."
}}

Texte :
\"\"\"
{context["documents"]}
\"\"\"
"""

prompt

'\nTu es un expert en pédagogie et tu dois créer une question de quiz à choix multiples (QCM) à partir du texte ci-dessous.\n\n**Important** : tu ne dois utiliser que les informations contenues dans le texte ci-dessous.  \n**Tu ne dois en aucun cas utiliser des connaissances extérieures.**\n\nContrainte :\n- La question doit porter sur une information claire et importante du texte.\n- Il doit y avoir exactement 4 choix.\n- Une seule bonne réponse.\n- Il faut que QCM porte uniquement\n- Retourne ta réponse **en JSON valide** dans ce format :\n\n{\n  "question": "...",\n  "choices": ["...", "...", "...", "..."],\n  "correct_answer": "..."\n}\n\nTexte :\n"""\n[\'LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  \\n \\n \\n1)  Qu’est-ce que l’Histoire ? \\n \\n*L’Histoire est l’étude de notre passé  pour mieux \\ncomprendre notre vie aujourd’hui. \\n \\n *Pour découvrir notre passé, les historiens font des fouilles \\narchéologiques, étudient des objets, des documents, des récits…  \\n*Ils rep

In [40]:
len(context['documents'])

81

In [41]:
from langchain.llms import Ollama
llm = Ollama(model="mistral", base_url="http://host.docker.internal:11434")
response = llm.invoke("Génère une question de QCM sur les rois de France.")
print(response)

 Question multiple choix sur les Rois de France :

Quel roi de France a régné pendant la Guerre de Cent Ans ?

A) Louis XI
B) Philippe VI
C) Charles VII
D) Louis IX

(Réponse : C) Charles VII)


In [42]:
#from langchain.llms import Ollama
#llm = Ollama(model="mistral", base_url="http://host.docker.internal:11434")
#response2 = llm.invoke(prompt)
#print(response2)

In [43]:
# NOTE
# Soit essayer une autre LLM que mistral via olamma
# Soit cleaner le code car le lancement serveur se fait dans le terminal ET le lancement de mistral ollama dans le terminal de la machine (et non dans le start)
# ollama serve -> terminal dev container
# ollama run mistral -> terminal machine

In [44]:
from langchain.chat_models import ChatOpenAI

load_dotenv()
open_router_api_key = os.getenv("OPEN_ROUTER_KEY")

llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=open_router_api_key,
    model="mistralai/mistral-7b-instruct"
)

response = llm.invoke(prompt)
print(response)

AuthenticationError: Error code: 401 - {'error': {'message': 'No auth credentials found', 'code': 401}}

In [45]:
llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPEN_ROUTER_KEY,
    model="mistralai/mistral-7b-instruct"
)

response = llm.invoke("Génère une question de QCM sur Napoléon. donne moi un JSON")
print(response)

NameError: name 'OPEN_ROUTER_KEY' is not defined

In [46]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPEN_ROUTER_KEY,
    model="mistralai/mistral-7b-instruct"
)

NameError: name 'OPEN_ROUTER_KEY' is not defined

In [47]:
json_quiz = []
i = 0
for doc in context['documents']:
    i+=1
    prompt_chunk = f"""
    Tu es un expert en pédagogie et tu dois créer une question de quiz à choix multiples (QCM) à partir du texte ci-dessous.

    **Important** : tu ne dois utiliser que les informations contenues dans le texte ci-dessous.  
    **Tu ne dois en aucun cas utiliser des connaissances extérieures.**

    Contrainte :
    - La question doit porter sur une information claire et importante du texte.
    - Il doit y avoir exactement 4 choix.
    - Une seule bonne réponse.
    - Il faut que QCM porte uniquement
    - Retourne ta réponse **en JSON valide** dans ce format :

    {{
    "question": "...",
    "choices": ["...", "...", "...", "..."],
    "correct_answer": "..."
    }}

    Texte :
    \"\"\"
    {doc}
    \"\"\"
    """

    response = llm.invoke(prompt_chunk)
    json_quiz.append(response)
    print(i, 'turn')

json_quiz

AuthenticationError: Error code: 401 - {'error': {'message': 'No auth credentials found', 'code': 401}}

In [48]:
json_quiz = []

for doc in range(0,20):
    prompt_chunk = f"""
    Tu es un expert en pédagogie et tu dois créer une question de quiz à choix multiples (QCM) à partir du texte ci-dessous.

    **Important** : tu ne dois utiliser que les informations contenues dans le texte ci-dessous.  
    **Tu ne dois en aucun cas utiliser des connaissances extérieures.**

    Contrainte :
    - La question doit porter sur une information claire et importante du texte.
    - Il doit y avoir exactement 4 choix.
    - Une seule bonne réponse.
    - Il faut que QCM porte uniquement
    - Retourne ta réponse **en JSON valide** dans ce format :

    {{
    "question": "...",
    "choices": ["...", "...", "...", "..."],
    "correct_answer": "..."
    }}

    Texte :
    \"\"\"
    {context['documents'][doc]}
    \"\"\"
    """

    response = llm.invoke(prompt_chunk)
    json_quiz.append(response)
    print(doc)

json_quiz

AuthenticationError: Error code: 401 - {'error': {'message': 'No auth credentials found', 'code': 401}}

In [49]:
context['documents'][4]

'font souvent la guerre.  \n \n*Ils vivent dans des fermes  regroupées en village. Ce sont des \nagriculteurs, des éleveurs , pour cela ils déboisent pour former des \nchamps qui sont encore présents aujourd’hui.  \n \n*Ce sont également des artisans  habiles qui créent des outils et \ninventent le tonneau, la roue, le savon, la charrue attelée, la faux…\n5) Qui sont les Gallo-Romains ? \n \n*Les Romains, emmenés par Jules César, ont envahi \nla Gaule en l’an 52 av. JC . Les Gaulois deviennent'

In [50]:
from langchain_community.chat_models import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(
    model_name="mixtral-8x7b-32768",  # Tu peux aussi tester llama3
    groq_api_key=groq_api_key
)

# Exemple de prompt simple
response = llm.invoke("Génère une question de quiz avec 4 réponses sur Napoléon.")
print(response)

ImportError: cannot import name 'ChatGroq' from 'langchain_community.chat_models' (/usr/local/lib/python3.11/site-packages/langchain_community/chat_models/__init__.py)

In [51]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-3.5-turbo",  # ou gpt-4o si tu veux tester
    temperature=0.3
)

prompt = "Crée une question à choix multiple sur la Révolution française avec 4 réponses dont une correcte."
response = llm.invoke(prompt)

print(response.content)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [52]:
from langchain_openai import ChatOpenAI
openai_api_key = os.getenv("OPENAI_KEY")

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
    api_key=OPENAI_KEY
)

response = llm.invoke("Explique-moi la Révolution française en 3 phrases simples.")
print(response.content)

NameError: name 'OPENAI_KEY' is not defined

In [53]:
from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    api_key=groq_api_key,  # ta clé Groq
    model="llama-3.1-8b-instant",  # ou llama‑3.1‑8b‑instant, llama3‑70b‑… etc.
    temperature=0.0,
    max_tokens=None,
    max_retries=2,
)

response = llm.invoke("donne moi une question de QCM sur les roi des France")
print(response.content)


Voici une question de QCM sur les rois de France :

Qui était le roi de France qui a été assassiné en 1419 et dont la mort a conduit à la guerre de Cent Ans ?

A) Philippe IV le Bel
B) Jean le Bon
C) Charles V le Sage
D) Charles VI le Fol

Quel est ton choix ?


# TEST VALIDE et optimal

In [54]:
json_quiz = []
i = 0

for doc in context['documents']:
    i+=1
    prompt_chunk = f"""
    Tu es un expert en pédagogie et tu dois créer une question de quiz à choix multiples (QCM) à partir du texte ci-dessous.

    **Important** : tu ne dois utiliser que les informations contenues dans le texte ci-dessous.  
    **Tu ne dois en aucun cas utiliser des connaissances extérieures.**

    Contrainte :
    - La question doit porter sur une information claire et importante du texte.
    - Il doit y avoir exactement 4 choix.
    - Une seule bonne réponse.
    - Il faut que QCM porte uniquement
    - **Réponds uniquement avec un objet JSON valide**, sans aucune explication, ni phrase introductive, ni conclusion.

    {{
        "question": "...",
        "choices": {{
            "a": "...",
            "b": "...",
            "c": "...",
            "d": "..."
        }},
        "correct_answer": {{
            "lettre": "...",
            "answer": "..."
        }}
    }}

    ### Exemple de chunck et résultat de QCM attendu
    #### Le chunk
    {{"En 1789, la Révolution française marque un tournant majeur dans l'histoire de France. Elle débute avec la convocation des États généraux par Louis XVI et conduit à l'abolition des privilèges ainsi qu'à la Déclaration des droits de l'homme et du citoyen. Cette période, marquée par une forte instabilité politique, voit l'émergence de nouvelles institutions et l'affirmation du principe de souveraineté nationale au détriment de la monarchie absolue."}}

    #### Le résultat
    {{
        "question": "Quel événement a marqué un tournant majeur dans l'histoire de France en 1789 ?",
        "choices": {{
            "a": "La chute de Napoléon",
            "b": "La Révolution française",
            "c": "La guerre franco-prussienne",
            "d": "Le couronnement de Louis XVI"
        }},
        "correct_answer": {{
            "lettre": "b",
            "answer": "La Révolution française"
        }}
    }}

    
    Texte :
    \"\"\"
    {doc}
    \"\"\"
    """

    response = llm.invoke(prompt_chunk)
    json_quiz.append(response)
    print(i)

json_quiz

1
2
3
4
5
6
7
8


KeyboardInterrupt: 

In [55]:
import json
json_quiz[0].content

'{\n    "question": "Qu\'est-ce que l\'Histoire ?",\n    "choices": {\n        "a": "L\'étude de notre présent pour comprendre notre avenir",\n        "b": "L\'étude de notre passé pour mieux comprendre notre vie aujourd\'hui",\n        "c": "La représentation du temps par une ligne graduée",\n        "d": "La période avant l\'invention de l\'écriture"\n    },\n    "correct_answer": {\n        "lettre": "b",\n        "answer": "L\'étude de notre passé pour mieux comprendre notre vie aujourd\'hui"\n    }\n}'

In [56]:
parsed = json.loads(json_quiz[0].content)
parsed

{'question': "Qu'est-ce que l'Histoire ?",
 'choices': {'a': "L'étude de notre présent pour comprendre notre avenir",
  'b': "L'étude de notre passé pour mieux comprendre notre vie aujourd'hui",
  'c': 'La représentation du temps par une ligne graduée',
  'd': "La période avant l'invention de l'écriture"},
 'correct_answer': {'lettre': 'b',
  'answer': "L'étude de notre passé pour mieux comprendre notre vie aujourd'hui"}}

In [57]:
len(context['ids'])

81